
# Лабораторная работа 2. Регрессия, мультиколлинеарность, множественные сравнения, байесовские выводы и ресемплинг

**Курс:** Прикладная статистика и анализ данных

**Раздел 2:** Статистическое моделирование, гипотезы и интерпретация    

**Цель:** построить и сравнить несколько спецификаций множественной линейной регрессии, диагностировать мультиколлинеарность и применить регуляризацию; корректно проверять семейства гипотез (контрасты/коэффициенты) с контролем FDR; провести байесовскую регрессию с априорами и постериорной проверкой предсказаний; оценить доверие к выводам с помощью бутстрепа и перестановочных тестов.

**Ожидаемые результаты:** уметь (1) задавать и интерпретировать линейные модели с взаимодействиями; (2) диагностировать и снижать мультиколлинеарность (VIF, Ridge/Lasso с подбором $\lambda$ по CV); (3) применять FDR-контроль к семействам проверок; (4) формулировать априоры и читать апостериор (PyMC), выполнять PPC; (5) строить бутстреп-ДИ и перестановочные p-значения; (6) оформлять воспроизводимый отчёт.



## 1. Данные и дизайн эксперимента

Требования к данным. Желательно $n\in[5\cdot10^4,1.5\cdot10^5]$, смешанные типы (число+категории+время). Разрешены большие наборы с честной подвыборкой (например, фиксированный интервал времени/случайная выборка 100–120 тыс. строк). Хорошими источниками являются городские такси (NYC TLC, Chicago Taxi), e-commerce (Olist), поведенческие/рейтинговые (MovieLens), дорожно-транспортные происшествия (STATS19).

Вопросы и переменные. Определите: целевую переменную $y$ (непрерывную: выручка/длительность/сумма), набор предикторов $X$ (стоимость, время суток, категория товара, погодные/районные признаки), потенциальные взаимодействия (например, «час$\times$район»). Разбейте данные на обучающую/валидационную схему с кросс-валидацией. Для задач на транзакциях важно исключить утечки (например, признаки, формирующиеся пост-фактум).

Критерии качества и риска. Выберите метрики: $R^2$, RMSE/MAE для базовой модели; для сравнения моделей используйте перекрёстную проверку (KFold/StratifiedKFold, при необходимости — TimeSeriesSplit). Фиксируйте случайные зерна и протоколы предобработки.


## 2. Линейные модели и ANOVA: спецификация и сравнение (минимум 600 слов)

OLS-модель. Запишите базовую спецификацию: $y = X\beta+\varepsilon$, где $\varepsilon\sim \mathcal{N}(0,\sigma^2 I)$. Оценка МНК: $\hat\beta=(X^\top X)^{-1}X^\top y$. Включите категориальные эффекты (OHE) и взаимодействия (например, «категория$\times$час»). Сравните вложенные модели через частичные F-тесты/девиансы.

ANOVA/контрасты. Для сравнения спецификаций используйте дисперсионный анализ в смысле разложения суммы квадратов и тестирования добавочных блоков признаков. Вектор контраста $c$ для проверки гипотезы $H_0: c^\top\beta=0$; оценка и стандартная ошибка выводятся из ковариационной матрицы $\widehat{\mathrm{Var}}(\hat\beta)=\hat\sigma^2 (X^\top X)^{-1}$.

Множественные проверки. При тестировании множества коэффициентов/контрастов применяйте контроль FDR (Benjamini–Hochberg): упорядочьте p-значения $p_{(1)}\le\cdots\le p_{(m)}$ и найдите максимальный $k$ с $p_{(k)}\le \frac{k}{m}\alpha$; значимыми считаются $p_{(1)},\dots,p_{(k)}$. Реализация — statsmodels.stats.multitest.multipletests.

## 3. Мультиколлинеарность, VIF и регуляризация (минимум 600 слов)

Диагностика. Индекс инфляции дисперсии для признака $j$: $VIF_j=\frac{1}{1-R_j^2}$, где $R_j^2$ — коэффициент детерминации регрессии $x_j$ на все прочие признаки. Большие $VIF$ (условно $>10$) указывают на нестабильность оценок и раздутые стандартные ошибки.
Ridge/Lasso. Регуляризованные оценки:
- Ridge: $\hat\beta_\lambda=\arg\min_\beta |y-X\beta|_2^2 + \lambda|\beta|_2^2$,
- Lasso: $\hat\beta_\lambda=\arg\min_\beta |y-X\beta|_2^2 + \lambda|\beta|_1$.
Подбирайте $\lambda$ по кросс-валидации (например, лог-сетка). Ridge снижает дисперсию оценок и стабилизирует прогнозы при умеренной корреляции; Lasso даёт разреженность и встроенный отбор признаков, но чувствителен к масштабированию. См. документы и примеры scikit-learn по OLS/Ridge и обобщениям.

## 4. Байесовская регрессия и постериорно-предсказательная проверка

Модель. Пусть $y\mid X,\beta,\sigma^2\sim \mathcal{N}(X\beta,\sigma^2 I)$, априорные распределения $p(\beta)$ и $p(\sigma^2)$ (например, $\beta\sim \mathcal{N}(0,\tau^2 I)$; $\sigma\sim \mathrm{HalfCauchy}$). Постериор: $p(\beta,\sigma^2\mid y)\propto p(y\mid X,\beta,\sigma^2),p(\beta),p(\sigma^2)$. Предсказательное распределение для нового $x_\star$: $p(y_\star\mid x_\star,y)=\int p(y_\star\mid x_\star,\beta,\sigma^2),p(\beta,\sigma^2\mid y),d\beta,d\sigma^2$.

Практика. В PyMC укажите модель в явном виде, используйте NUTS для сэмплинга, проверьте сходимость (R-hat, ESS), постериорные интервалые для коэффициентов и PPC для согласованности с наблюдаемыми данными.

## 5. Ресемплинг: бутстреп-ДИ и перестановочные тесты

Бутстреп. Для параметра $\theta=g(X,y)$ (например, коэффициента $\beta_j$ или RMSE) сформируйте $B$ бутстреп-выборок с возвращением, получите ${\hat\theta^{(b)}}{b=1}^B$, и задайте ДИ по процентилям: $[q{\alpha/2},q_{1-\alpha/2}]$.

Перестановочные тесты. Для статистики $T$ (например, корреляция/разность RMSE между моделями на одинаковых фолдах) переставляйте метки/знаки по нулевой гипотезе, получая ${T^{(b)}}$, и вычисляйте $p$-значение $p=\frac{1+\sum_{b=1}^B \mathbb{I}(|T^{(b)}|\ge |T_{\text{obs}}|)}{1+B}$. Эти процедуры особенно полезны при нарушениях гауссовости/гомоскедастичности либо сложной зависимой структуре данных.

## Пул рекомендованных датасетов

D1. NYC TLC Trip Records (Yellow/Green). Возьмите один месяц и случайную подвыборку 100–120k поездок (например, январь 2019). Документация и структура полей — в руководстве TLC.

D2. Chicago Taxi Trips. Фильтрация по кварталу 2020-го даёт нужный объём; таблица содержит стоимость, время, гео-идентификаторы.

D3. Olist Brazilian E-commerce. Таблица orders $\approx 99,441$ заказов + связи с продуктами/категориями/логистикой. Отлично подходит для регрессии по выручке/времени доставки.

D4. MovieLens 100K. 100k рейтингов с признаками пользователей/фильмов; можно строить регрессию по рейтингу, учитывая жанры и время.

D5. UK Road Safety (STATS19). Выберите один год (например, 2017) и сформируйте подмножество ≈100–140k инцидентов, обогащая погодой/дорожными условиями.

## 35 вариантов заданий (обобщающие, с привязкой к D1–D5)
### Общие методические требования (для всех вариантов):
1) Постройте базовую OLS-модель (целевую переменную укажу в каждом варианте).
2) Выполните диагностику мультиколлинеарности (VIF) и сравните с Ridge/Lasso (подбор $\lambda$ по $K$-fold CV).
3) Проведите проверки гипотез по коэффициентам/контрастам; множественные сравнения контролируйте по BH-FDR.
4) Постройте байесовскую версию ключевой модели с умеренно информативными априорами и выполните предиктивную проверку (PPC).
5) Оцените устойчивость: бутстреп-ДИ для ключевого показателя (RMSE/MAE/$R^2$) и перестановочный тест для сопоставления двух моделей.
6) Объём данных: используйте подвыборку 50–150 тыс. строк из указанного источника, обоснуйте схему семплинга (по месяцам/зонам/времени и т. п.).

MovieLens-Rating (D4). Цель: предсказать rating. Признаки: жанровые фиктивные переменные (из movies.csv), средний рейтинг фильма (leave-one-out), активность пользователя (кол-во оценок; LOO), временной тренд по году. Сравните OLS vs. Ridge; выполните BH-FDR по семейству жанров. Бутстреп-ДИ для $R^2$; перестановка для разницы RMSE между OLS/Ridge; байес-версия с частичным пуллингом по фильмам/пользователям (иерархичность).

In [1]:
# Импорты и фиксируем зерно случайности для воспроизводимости
import os
import sys
import math
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime

from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Убедимся, что рабочая директория — папка с ноутбуком
print("CWD =", os.getcwd())


CWD = C:\Users\batar\Python\Applied_statistics_master_degree\Lab_2


In [2]:
# Пути (адаптируются, если ноутбук лежит в C:...\Lab_2\)
data_dir = Path("ml-100k")

path_ratings = data_dir / "u.data"
path_items   = data_dir / "u.item"
path_users   = data_dir / "u.user"   # пока не используем, но оставим для расширений

# Загрузка
ratings = pd.read_csv(
    path_ratings, sep="\t", header=None,
    names=["user_id", "movie_id", "rating", "timestamp"]
)

items = pd.read_csv(
    path_items, sep="|", header=None, encoding="latin-1",
    names=[
        "movie_id","title","release_date","video_release","imdb_url",
        "unknown","Action","Adventure","Animation","Children's","Comedy","Crime",
        "Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery",
        "Romance","Sci-Fi","Thriller","War","Western"
    ]
)

print(ratings.shape, items.shape)
ratings.head(), items.head(2)


(100000, 4) (1682, 24)


(   user_id  movie_id  rating  timestamp
 0      196       242       3  881250949
 1      186       302       3  891717742
 2       22       377       1  878887116
 3      244        51       2  880606923
 4      166       346       1  886397596,
    movie_id             title release_date  video_release  \
 0         1  Toy Story (1995)  01-Jan-1995            NaN   
 1         2  GoldenEye (1995)  01-Jan-1995            NaN   
 
                                             imdb_url  unknown  Action  \
 0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
 1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
 
    Adventure  Animation  Children's  ...  Fantasy  Film-Noir  Horror  Musical  \
 0          0          1           1  ...        0          0       0        0   
 1          1          0           0  ...        0          0       0        0   
 
    Mystery  Romance  Sci-Fi  Thriller  War  Western  
 0        0        0       0         0

In [3]:
# Парсим год релиза из строки "dd-Mmm-YYYY", иногда есть пропуски
def parse_year_from_date(s):
    if isinstance(s, str) and len(s) >= 4:
        try:
            return int(s[-4:])
        except Exception:
            return np.nan
    return np.nan

items["year_release"] = items["release_date"].apply(parse_year_from_date)
genre_cols = [
    "unknown","Action","Adventure","Animation","Children's","Comedy","Crime",
    "Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery",
    "Romance","Sci-Fi","Thriller","War","Western"
]

# Превратим timestamp оценок в календарный год — это "временной тренд по году" с точки зрения момента оценки
ratings["dt"] = pd.to_datetime(ratings["timestamp"], unit="s", utc=True)
ratings["year_rate"] = ratings["dt"].dt.year.astype(int)

# Склеиваем
df = ratings.merge(
    items[["movie_id", "year_release"] + genre_cols],
    on="movie_id", how="left"
)

# Обработка пропусков года релиза (редки): заменим на медиану
df["year_release"] = df["year_release"].fillna(df["year_release"].median())

# Центрирование временных признаков (ортогонализует сдвиг)
df["year_release_c"] = df["year_release"] - df["year_release"].mean()
df["year_rate_c"]    = df["year_rate"]    - df["year_rate"].mean()

# Контроль размера и базовые sanity-checks
print("Rows:", len(df))
print("Users:", df["user_id"].nunique(), "Movies:", df["movie_id"].nunique())
print("Years (rate):", df["year_rate"].min(), "–", df["year_rate"].max())
df.head()


Rows: 100000
Users: 943 Movies: 1682
Years (rate): 1997 – 1998


,user_id,movie_id,rating,timestamp,dt,year_rate,year_release,unknown,Action,Adventure,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year_release_c,year_rate_c
0,196,242,3,881250949,1997-12-04 15:55:49+00:00,1997,1997.0,0,0,0,...,0,0,0,0,0,0,0,0,9.04324,-0.47101
1,186,302,3,891717742,1998-04-04 19:22:22+00:00,1998,1997.0,0,0,0,...,0,0,1,0,0,1,0,0,9.04324,0.52899
2,22,377,1,878887116,1997-11-07 07:18:36+00:00,1997,1994.0,0,0,0,...,0,0,0,0,0,0,0,0,6.04324,-0.47101
3,244,51,2,880606923,1997-11-27 05:02:03+00:00,1997,1994.0,0,0,0,...,0,0,0,1,0,0,1,1,6.04324,-0.47101
4,166,346,1,886397596,1998-02-02 05:33:16+00:00,1998,1997.0,0,0,0,...,0,0,0,0,0,0,0,0,9.04324,0.52899


In [4]:
# Целевая переменная
y_col = "rating"

# Базовые предикторы: жанры + временные тренды
# (LOO-признаки добавим позже в разделе фичеинжиниринга, чтобы не допустить утечек)
base_feature_cols = genre_cols + ["year_release_c", "year_rate_c"]

# Пример потенциальных взаимодействий для следующих пунктов (сейчас только объявляем):
# - Жанр × год оценки (динамика восприятия жанра)
# - Жанр × год релиза (устаревание эффекта)
# Эти взаимодействия создадим позже осознанно, когда будем смотреть VIF/регуляризацию.
potential_interactions = {
    "genre_x_year_rate": [("year_rate_c", g) for g in genre_cols],
    "genre_x_year_release": [("year_release_c", g) for g in genre_cols],
}

print("Target:", y_col)
print("Base features (count):", len(base_feature_cols))
base_feature_cols[:8], "...", base_feature_cols[-5:]


Target: rating
Base features (count): 21


(['unknown',
  'Action',
  'Adventure',
  'Animation',
  "Children's",
  'Comedy',
  'Crime',
  'Documentary'],
 '...',
 ['Thriller', 'War', 'Western', 'year_release_c', 'year_rate_c'])

In [5]:
# Для рейтингов критично: пользовательские паттерны.
# Чтобы не «подсматривать» индивидуальные эффекты, держим пользователей разнесёнными по фолдам.
# Это снижает optimistic bias и имитирует генерализацию на новых пользователей.

n_splits = 5
cv_splitter = GroupKFold(n_splits=n_splits)
cv_groups = df["user_id"].values  # группы = пользователи

# Пример: посчитаем размер фолдов (без обучения — просто дизайн)
fold_sizes = []
for fold_id, (_, te_idx) in enumerate(cv_splitter.split(df, groups=cv_groups), 1):
    fold_sizes.append(len(te_idx))
    print(f"Fold {fold_id}: test size = {len(te_idx)}")

print("Sum test sizes =", sum(fold_sizes), "==", len(df))


Fold 1: test size = 20008
Fold 2: test size = 20008
Fold 3: test size = 20008
Fold 4: test size = 19988
Fold 5: test size = 19988
Sum test sizes = 100000 == 100000


In [6]:
def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def r2(y_true, y_pred):
    return r2_score(y_true, y_pred)

metrics = {
    "RMSE": rmse,
    "MAE": mae,
    "R2": r2
}

print("Метрики готовы:", list(metrics.keys()))
print("Random state =", RANDOM_STATE)


Метрики готовы: ['RMSE', 'MAE', 'R2']
Random state = 42


In [7]:
# На этом шаге мы НЕ создаём LOO-признаки, чтобы не спутать дизайн и вычисления.
# Здесь — только базовый X для первичной диагностики и профилирования распределений.

X_base = df[base_feature_cols].astype(float).copy()
y = df[y_col].astype(float).values

print("X_base shape:", X_base.shape, "y shape:", y.shape)
print("Пример жанровых частот:")
df[genre_cols].mean().sort_values(ascending=False).head(10)


X_base shape: (100000, 21) y shape: (100000,)
Пример жанровых частот:


Drama         0.39895
Comedy        0.29832
Action        0.25589
Thriller      0.21872
Romance       0.19461
Adventure     0.13753
Sci-Fi        0.12730
War           0.09398
Crime         0.08055
Children's    0.07182
dtype: float64

In [8]:
n = len(df)
n_users = df["user_id"].nunique()
n_movies = df["movie_id"].nunique()

assert 50_000 <= n <= 150_000, "Размер данных не попадает в требуемый диапазон."
print(f"OK: n={n} в требуемом диапазоне [50k, 150k]. Users={n_users}, Movies={n_movies}")

# Смешанные типы подтверждаем:
has_numeric = True
has_categorical_like = len(genre_cols) > 0
has_temporal = True  # есть year_rate и year_release
print(f"Типы: numeric={has_numeric}, categorical(one-hot)={has_categorical_like}, temporal={has_temporal}")


OK: n=100000 в требуемом диапазоне [50k, 150k]. Users=943, Movies=1682
Типы: numeric=True, categorical(one-hot)=True, temporal=True


In [9]:
df.head(5)

,user_id,movie_id,rating,timestamp,dt,year_rate,year_release,unknown,Action,Adventure,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year_release_c,year_rate_c
0,196,242,3,881250949,1997-12-04 15:55:49+00:00,1997,1997.0,0,0,0,...,0,0,0,0,0,0,0,0,9.04324,-0.47101
1,186,302,3,891717742,1998-04-04 19:22:22+00:00,1998,1997.0,0,0,0,...,0,0,1,0,0,1,0,0,9.04324,0.52899
2,22,377,1,878887116,1997-11-07 07:18:36+00:00,1997,1994.0,0,0,0,...,0,0,0,0,0,0,0,0,6.04324,-0.47101
3,244,51,2,880606923,1997-11-27 05:02:03+00:00,1997,1994.0,0,0,0,...,0,0,0,1,0,0,1,1,6.04324,-0.47101
4,166,346,1,886397596,1998-02-02 05:33:16+00:00,1998,1997.0,0,0,0,...,0,0,0,0,0,0,0,0,9.04324,0.52899


## OLS МОДЕЛИ И ANOVA

In [ ]:
# =============================================================================
# OLS МОДЕЛИ И ANOVA
# =============================================================================

def fit_ols_model(
    X: pd.DataFrame,
    y: np.ndarray,
    add_constant: bool = True
) -> sm.regression.linear_model.RegressionResultsWrapper:
    """
    Построить OLS модель

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    y : np.ndarray
        Целевая переменная
    add_constant : bool
        Добавить константу

    Returns
    -------
    statsmodels RegressionResults
        Результаты регрессии
    """
    # TODO: Добавить константу если нужно
    # TODO: Обучить OLS модель через statsmodels
    # TODO: Вывести summary
    pass


def compare_nested_models(
    model_restricted: sm.regression.linear_model.RegressionResultsWrapper,
    model_full: sm.regression.linear_model.RegressionResultsWrapper,
    alpha: float = 0.05
) -> Dict[str, float]:
    """
    Сравнить вложенные модели через F-тест

    Parameters
    ----------
    model_restricted : RegressionResults
        Ограниченная модель
    model_full : RegressionResults
        Полная модель
    alpha : float
        Уровень значимости

    Returns
    -------
    Dict[str, float]
        Статистики теста (F-stat, p-value, conclusion)
    """
    # TODO: Вычислить F-статистику
    # TODO: Сравнить RSS моделей
    # TODO: Вернуть результат теста
    pass


def test_linear_contrasts(
    model: sm.regression.linear_model.RegressionResultsWrapper,
    contrast_matrix: np.ndarray,
    contrast_names: Optional[List[str]] = None
) -> pd.DataFrame:
    """
    Проверить линейные контрасты

    Parameters
    ----------
    model : RegressionResults
        Обученная модель
    contrast_matrix : np.ndarray
        Матрица контрастов (каждая строка - один контраст)
    contrast_names : Optional[List[str]]
        Названия контрастов

    Returns
    -------
    pd.DataFrame
        Результаты тестов контрастов
    """
    # TODO: Для каждого контраста c^T β = 0
    # TODO: Вычислить статистику, p-value
    # TODO: Вернуть DataFrame с результатами
    pass


def compute_anova_table(
    model: sm.regression.linear_model.RegressionResultsWrapper,
    typ: int = 2
) -> pd.DataFrame:
    """
    Вычислить таблицу ANOVA

    Parameters
    ----------
    model : RegressionResults
        Обученная модель
    typ : int
        Тип ANOVA (1, 2, или 3)

    Returns
    -------
    pd.DataFrame
        Таблица ANOVA
    """
    # TODO: Использовать statsmodels для построения ANOVA
    # TODO: Вернуть таблицу с SS, df, F, p-value
    pass

In [ ]:
# =============================================================================
# МУЛЬТИКОЛЛИНЕАРНОСТЬ И VIF
# =============================================================================

def compute_vif(X: pd.DataFrame, add_constant: bool = True) -> pd.DataFrame:
    """
    Вычислить VIF для всех признаков

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    add_constant : bool
        Добавить константу перед вычислением

    Returns
    -------
    pd.DataFrame
        VIF для каждого признака
    """
    # TODO: Для каждого признака вычислить VIF_j = 1/(1-R²_j)
    # TODO: Вернуть DataFrame с признаками и их VIF
    # TODO: Отметить признаки с VIF > 10
    pass


def diagnose_multicollinearity(
    X: pd.DataFrame,
    vif_threshold: float = 10.0,
    corr_threshold: float = 0.8
) -> Dict[str, Union[pd.DataFrame, List[Tuple[str, str]]]]:
    """
    Диагностировать мультиколлинеарность

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    vif_threshold : float
        Порог VIF
    corr_threshold : float
        Порог корреляции

    Returns
    -------
    Dict
        VIF таблица и пары сильно коррелирующих признаков
    """
    # TODO: Вычислить VIF
    # TODO: Найти пары признаков с высокой корреляцией
    # TODO: Рекомендовать какие признаки удалить
    pass


def iteratively_remove_high_vif(
    X: pd.DataFrame,
    vif_threshold: float = 10.0,
    max_iterations: int = 10
) -> Tuple[pd.DataFrame, List[str]]:
    """
    Итеративно удалять признаки с высоким VIF

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    vif_threshold : float
        Порог VIF
    max_iterations : int
        Максимум итераций

    Returns
    -------
    X_reduced : pd.DataFrame
        Редуцированная матрица признаков
    removed_features : List[str]
        Список удалённых признаков
    """
    # TODO: Итеративно вычислять VIF
    # TODO: На каждой итерации удалять признак с максимальным VIF > threshold
    # TODO: Остановиться когда все VIF < threshold
    pass

In [ ]:
# =============================================================================
# РЕГУЛЯРИЗАЦИЯ (RIDGE/LASSO)
# =============================================================================

def fit_ridge_cv(
    X: pd.DataFrame,
    y: np.ndarray,
    alphas: Optional[np.ndarray] = None,
    cv: int = 5,
    scoring: str = 'neg_mean_squared_error'
) -> Tuple[Ridge, np.ndarray, float]:
    """
    Подобрать Ridge регрессию с кросс-валидацией

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    y : np.ndarray
        Целевая переменная
    alphas : Optional[np.ndarray]
        Сетка гиперпараметров λ
    cv : int
        Число фолдов
    scoring : str
        Метрика для оптимизации

    Returns
    -------
    model : Ridge
        Обученная модель с лучшим α
    alphas : np.ndarray
        Использованные значения α
    best_alpha : float
        Оптимальное значение α
    """
    # TODO: Стандартизировать признаки
    # TODO: Использовать RidgeCV для подбора α
    # TODO: Построить график CV score vs alpha
    # TODO: Вернуть лучшую модель
    pass


def fit_lasso_cv(
    X: pd.DataFrame,
    y: np.ndarray,
    alphas: Optional[np.ndarray] = None,
    cv: int = 5,
    scoring: str = 'neg_mean_squared_error',
    max_iter: int = 10000
) -> Tuple[Lasso, np.ndarray, float]:
    """
    Подобрать Lasso регрессию с кросс-валидацией

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    y : np.ndarray
        Целевая переменная
    alphas : Optional[np.ndarray]
        Сетка гиперпараметров λ
    cv : int
        Число фолдов
    scoring : str
        Метрика для оптимизации
    max_iter : int
        Максимум итераций

    Returns
    -------
    model : Lasso
        Обученная модель с лучшим α
    alphas : np.ndarray
        Использованные значения α
    best_alpha : float
        Оптимальное значение α
    """
    # TODO: Стандартизировать признаки
    # TODO: Использовать LassoCV для подбора α
    # TODO: Построить график CV score vs alpha
    # TODO: Построить график путей коэффициентов (Lasso path)
    # TODO: Вернуть лучшую модель
    pass


def compare_regularization_methods(
    X_train: pd.DataFrame,
    y_train: np.ndarray,
    X_test: pd.DataFrame,
    y_test: np.ndarray
) -> pd.DataFrame:
    """
    Сравнить OLS, Ridge и Lasso

    Parameters
    ----------
    X_train, y_train : обучающая выборка
    X_test, y_test : тестовая выборка

    Returns
    -------
    pd.DataFrame
        Таблица сравнения метрик (RMSE, MAE, R²)
    """
    # TODO: Обучить OLS, Ridge, Lasso
    # TODO: Вычислить метрики на train и test
    # TODO: Сравнить количество ненулевых коэффициентов
    # TODO: Вернуть сводную таблицу
    pass


def plot_coefficient_comparison(
    models: Dict[str, Union[LinearRegression, Ridge, Lasso]],
    feature_names: List[str],
    top_n: int = 20
) -> None:
    """
    Визуализировать сравнение коэффициентов моделей

    Parameters
    ----------
    models : Dict[str, model]
        Словарь обученных моделей
    feature_names : List[str]
        Названия признаков
    top_n : int
        Показать top N признаков по абсолютной величине
    """
    # TODO: Извлечь коэффициенты из каждой модели
    # TODO: Построить grouped bar plot
    # TODO: Отсортировать по важности
    pass

In [ ]:
# =============================================================================
# МНОЖЕСТВЕННЫЕ СРАВНЕНИЯ И FDR-КОНТРОЛЬ
# =============================================================================

def apply_fdr_correction(
    pvalues: np.ndarray,
    alpha: float = 0.05,
    method: str = 'fdr_bh'
) -> Tuple[np.ndarray, np.ndarray, float]:
    """
    Применить коррекцию FDR (Benjamini-Hochberg)

    Parameters
    ----------
    pvalues : np.ndarray
        Массив p-значений
    alpha : float
        Уровень FDR
    method : str
        Метод коррекции ('fdr_bh', 'fdr_by', 'bonferroni')

    Returns
    -------
    rejected : np.ndarray
        Булевы маски отклонённых гипотез
    pvals_corrected : np.ndarray
        Скорректированные p-значения
    alphac : float
        Скорректированный уровень значимости
    """
    # TODO: Использовать statsmodels.stats.multitest.multipletests
    # TODO: Вернуть результаты коррекции
    pass


def test_coefficient_family(
    model: sm.regression.linear_model.RegressionResultsWrapper,
    family_indices: List[int],
    family_name: str,
    fdr_alpha: float = 0.05
) -> pd.DataFrame:
    """
    Проверить семейство коэффициентов с FDR-контролем

    Parameters
    ----------
    model : RegressionResults
        Обученная модель
    family_indices : List[int]
        Индексы коэффициентов в семействе
    family_name : str
        Название семейства
    fdr_alpha : float
        Уровень FDR

    Returns
    -------
    pd.DataFrame
        Результаты тестов с коррекцией
    """
    # TODO: Извлечь p-значения для семейства
    # TODO: Применить FDR-коррекцию
    # TODO: Вернуть таблицу с результатами
    pass


def test_multiple_contrasts(
    model: sm.regression.linear_model.RegressionResultsWrapper,
    contrast_dict: Dict[str, np.ndarray],
    fdr_alpha: float = 0.05
) -> pd.DataFrame:
    """
    Проверить множество контрастов с FDR-контролем

    Parameters
    ----------
    model : RegressionResults
        Обученная модель
    contrast_dict : Dict[str, np.ndarray]
        Словарь {название: вектор контраста}
    fdr_alpha : float
        Уровень FDR

    Returns
    -------
    pd.DataFrame
        Результаты всех контрастов с коррекцией
    """
    # TODO: Для каждого контраста вычислить статистику и p-value
    # TODO: Применить FDR-коррекцию к семейству p-values
    # TODO: Вернуть сводную таблицу
    pass


def visualize_multiple_testing_results(
    pvalues_raw: np.ndarray,
    pvalues_corrected: np.ndarray,
    rejected: np.ndarray,
    test_names: List[str],
    alpha: float = 0.05
) -> None:
    """
    Визуализировать результаты множественных сравнений

    Parameters
    ----------
    pvalues_raw : np.ndarray
        Исходные p-значения
    pvalues_corrected : np.ndarray
        Скорректированные p-значения
    rejected : np.ndarray
        Маска отклонённых гипотез
    test_names : List[str]
        Названия тестов
    alpha : float
        Уровень значимости
    """
    # TODO: Построить volcano plot или bar plot
    # TODO: Показать линию отсечения α
    # TODO: Выделить значимые тесты
    pass



In [ ]:
# =============================================================================
# БАЙЕСОВСКАЯ РЕГРЕССИЯ (PyMC)
# =============================================================================

def build_bayesian_linear_model(
    X: pd.DataFrame,
    y: np.ndarray,
    prior_scale: float = 10.0,
    prior_sigma: str = 'halfcauchy'
) -> pm.Model:
    """
    Построить байесовскую линейную модель

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков (стандартизированная)
    y : np.ndarray
        Целевая переменная
    prior_scale : float
        Масштаб априорного распределения для β
    prior_sigma : str
        Тип априора для σ ('halfcauchy', 'halfnormal', 'exponential')

    Returns
    -------
    pm.Model
        PyMC модель
    """
    # TODO: Создать PyMC модель
    # TODO: Задать априоры: β ~ N(0, prior_scale²)
    # TODO: Задать априор для σ (HalfCauchy, HalfNormal)
    # TODO: Задать likelihood: y ~ N(Xβ, σ²)
    pass


def sample_bayesian_model(
    model: pm.Model,
    draws: int = 2000,
    tune: int = 1000,
    chains: int = 4,
    target_accept: float = 0.95
) -> az.InferenceData:
    """
    Сэмплировать из апостериорного распределения

    Parameters
    ----------
    model : pm.Model
        PyMC модель
    draws : int
        Число сэмплов после warmup
    tune : int
        Число warmup итераций
    chains : int
        Число цепей
    target_accept : float
        Целевая вероятность принятия (для NUTS)

    Returns
    -------
    az.InferenceData
        Результаты сэмплирования
    """
    # TODO: Использовать pm.sample с NUTS
    # TODO: Вернуть InferenceData объект
    pass


def check_convergence_diagnostics(trace: az.InferenceData) -> pd.DataFrame:
    """
    Проверить диагностики сходимости

    Parameters
    ----------
    trace : az.InferenceData
        Результаты сэмплирования

    Returns
    -------
    pd.DataFrame
        Таблица с R-hat, ESS для каждого параметра
    """
    # TODO: Вычислить R-hat (должен быть < 1.01)
    # TODO: Вычислить ESS (effective sample size)
    # TODO: Построить trace plots
    # TODO: Построить autocorrelation plots
    pass


def posterior_predictive_check(
    model: pm.Model,
    trace: az.InferenceData,
    y_observed: np.ndarray,
    n_samples: int = 1000
) -> None:
    """
    Выполнить постериорно-предсказательную проверку (PPC)

    Parameters
    ----------
    model : pm.Model
        PyMC модель
    trace : az.InferenceData
        Апостериорное распределение
    y_observed : np.ndarray
        Наблюдаемые данные
    n_samples : int
        Число предсказательных сэмплов
    """
    # TODO: Сгенерировать предсказания из апостериора
    # TODO: Сравнить распределение предсказаний с наблюдениями
    # TODO: Построить PPC plots (гистограммы, density plots)
    # TODO: Проверить статистики (среднее, дисперсия, квантили)
    pass


def extract_posterior_intervals(
    trace: az.InferenceData,
    var_names: List[str],
    hdi_prob: float = 0.95
) -> pd.DataFrame:
    """
    Извлечь апостериорные интервалы для параметров

    Parameters
    ----------
    trace : az.InferenceData
        Апостериорное распределение
    var_names : List[str]
        Названия переменных
    hdi_prob : float
        Вероятность для HDI (highest density interval)

    Returns
    -------
    pd.DataFrame
        Таблица с mean, sd, HDI для каждого параметра
    """
    # TODO: Использовать arviz.summary
    # TODO: Извлечь HDI интервалы
    # TODO: Построить forest plots
    pass


def compare_bayesian_models(
    models: Dict[str, pm.Model],
    traces: Dict[str, az.InferenceData]
) -> pd.DataFrame:
    """
    Сравнить байесовские модели через LOO/WAIC

    Parameters
    ----------
    models : Dict[str, pm.Model]
        Словарь моделей
    traces : Dict[str, az.InferenceData]
        Словарь апостериорных распределений

    Returns
    -------
    pd.DataFrame
        Таблица сравнения (LOO, WAIC, weights)
    """
    # TODO: Вычислить LOO (leave-one-out cross-validation)
    # TODO: Вычислить WAIC
    # TODO: Использовать arviz.compare
    # TODO: Вернуть таблицу сравнения моделей
    pass



In [ ]:
# =============================================================================
# 7. БУТСТРЕП
# =============================================================================

def bootstrap_metric(
    X: pd.DataFrame,
    y: np.ndarray,
    metric_func: callable,
    n_bootstrap: int = 1000,
    confidence_level: float = 0.95,
    random_state: int = RANDOM_STATE
) -> Dict[str, float]:
    """
    Вычислить бутстреп доверительный интервал для метрики

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    y : np.ndarray
        Целевая переменная
    metric_func : callable
        Функция метрики (принимает X, y; возвращает число)
    n_bootstrap : int
        Число бутстреп-итераций
    confidence_level : float
        Уровень доверия
    random_state : int
        Зерно для воспроизводимости

    Returns
    -------
    Dict[str, float]
        mean, std, CI_lower, CI_upper
    """
    # TODO: Для каждой итерации:
    #   - Сформировать бутстреп-выборку (с возвращением)
    #   - Вычислить метрику
    # TODO: Вычислить процентильный CI
    # TODO: Построить гистограмму распределения
    pass


def bootstrap_coefficients(
    X: pd.DataFrame,
    y: np.ndarray,
    n_bootstrap: int = 1000,
    confidence_level: float = 0.95,
    model_type: str = 'ols'
) -> pd.DataFrame:
    """
    Бутстреп доверительные интервалы для коэффициентов регрессии

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    y : np.ndarray
        Целевая переменная
    n_bootstrap : int
        Число бутстреп-итераций
    confidence_level : float
        Уровень доверия
    model_type : str
        Тип модели ('ols', 'ridge', 'lasso')

    Returns
    -------
    pd.DataFrame
        Таблица с оценками и CI для каждого коэффициента
    """
    # TODO: Для каждой итерации обучить модель на бутстреп-выборке
    # TODO: Сохранить коэффициенты
    # TODO: Вычислить процентильные CI для каждого коэффициента
    # TODO: Построить violin plots для важных коэффициентов
    pass


def bootstrap_model_comparison(
    X: pd.DataFrame,
    y: np.ndarray,
    model1_func: callable,
    model2_func: callable,
    metric_func: callable,
    n_bootstrap: int = 1000,
    confidence_level: float = 0.95
) -> Dict[str, Union[float, np.ndarray]]:
    """
    Бутстреп сравнение двух моделей

    Parameters
    ----------
    X, y : данные
    model1_func, model2_func : callable
        Функции обучения моделей
    metric_func : callable
        Функция метрики
    n_bootstrap : int
        Число итераций
    confidence_level : float
        Уровень доверия

    Returns
    -------
    Dict
        Статистики разницы метрик и CI
    """
    # TODO: Для каждой итерации:
    #   - Сформировать бутстреп-выборку
    #   - Обучить обе модели
    #   - Вычислить разницу метрик
    # TODO: Вычислить CI для разницы
    # TODO: Тест: включает ли CI ноль?
    pass



In [ ]:
# =============================================================================
# ПЕРЕСТАНОВОЧНЫЕ ТЕСТЫ
# =============================================================================

def permutation_test(
    X: pd.DataFrame,
    y: np.ndarray,
    test_statistic_func: callable,
    n_permutations: int = 1000,
    alternative: str = 'two-sided',
    random_state: int = RANDOM_STATE
) -> Dict[str, float]:
    """
    Перестановочный тест для статистики

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    y : np.ndarray
        Целевая переменная
    test_statistic_func : callable
        Функция вычисления статистики (принимает X, y)
    n_permutations : int
        Число перестановок
    alternative : str
        'two-sided', 'greater', 'less'
    random_state : int
        Зерно

    Returns
    -------
    Dict[str, float]
        observed_statistic, p_value, permuted_statistics
    """
    # TODO: Вычислить наблюдаемую статистику
    # TODO: Для каждой перестановки:
    #   - Перемешать y (или X, в зависимости от теста)
    #   - Вычислить статистику на перемешанных данных
    # TODO: Вычислить p-value: (1 + #{|T_perm| >= |T_obs|}) / (1 + n_perm)
    # TODO: Построить гистограмму нулевого распределения
    pass


def permutation_test_model_difference(
    X: pd.DataFrame,
    y: np.ndarray,
    model1_func: callable,
    model2_func: callable,
    metric_func: callable,
    n_permutations: int = 1000,
    cv_folds: int = 5
) -> Dict[str, float]:
    """
    Перестановочный тест для разницы метрик двух моделей

    Parameters
    ----------
    X, y : данные
    model1_func, model2_func : callable
        Функции обучения моделей
    metric_func : callable
        Функция метрики
    n_permutations : int
        Число перестановок
    cv_folds : int
        Число фолдов для CV

    Returns
    -------
    Dict[str, float]
        observed_diff, p_value
    """
    # TODO: Вычислить разницу метрик на исходных данных (с CV)
    # TODO: Для каждой перестановки:
    #   - Перемешать y
    #   - Вычислить разницу метрик с CV
    # TODO: Вычислить p-value
    pass


def permutation_feature_importance(
    X: pd.DataFrame,
    y: np.ndarray,
    model_func: callable,
    metric_func: callable,
    n_permutations: int = 100
) -> pd.DataFrame:
    """
    Вычислить важность признаков через перестановки

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    y : np.ndarray
        Целевая переменная
    model_func : callable
        Функция обучения модели
    metric_func : callable
        Функция метрики (чем выше, тем лучше)
    n_permutations : int
        Число перестановок для каждого признака

    Returns
    -------
    pd.DataFrame
        Важность каждого признака (снижение метрики)
    """
    # TODO: Обучить базовую модель, вычислить базовую метрику
    # TODO: Для каждого признака:
    #   - Перемешать значения признака
    #   - Вычислить метрику с перемешанным признаком
    #   - Важность = (base_metric - permuted_metric)
    # TODO: Усреднить по перестановкам
    # TODO: Построить bar plot важности
    pass

In [ ]:
# =============================================================================
# КРОСС-ВАЛИДАЦИЯ И ОЦЕНКА МОДЕЛЕЙ
# =============================================================================

def cross_validate_models(
    X: pd.DataFrame,
    y: np.ndarray,
    models: Dict[str, callable],
    cv: int = 5,
    scoring: List[str] = ['neg_mean_squared_error', 'r2']
) -> pd.DataFrame:
    """
    Кросс-валидация нескольких моделей

    Parameters
    ----------
    X : pd.DataFrame
        Матрица признаков
    y : np.ndarray
        Целевая переменная
    models : Dict[str, callable]
        Словарь {название: модель}
    cv : int
        Число фолдов
    scoring : List[str]
        Список метрик

    Returns
    -------
    pd.DataFrame
        Результаты CV для каждой модели и метрики
    """
    # TODO: Для каждой модели выполнить cross_val_score
    # TODO: Вычислить среднее и std по фолдам
    # TODO: Вернуть сводную таблицу
    pass


def evaluate_model_on_test(
    model,
    X_train: pd.DataFrame,
    y_train: np.ndarray,
    X_test: pd.DataFrame,
    y_test: np.ndarray
) -> Dict[str, float]:
    """
    Оценить модель на тестовой выборке

    Parameters
    ----------
    model : обучаемая модель
    X_train, y_train : обучающая выборка
    X_test, y_test : тестовая выборка

    Returns
    -------
    Dict[str, float]
        Метрики (RMSE, MAE, R², MAPE)
    """
    # TODO: Обучить модель на train
    # TODO: Сделать предсказания на test
    # TODO: Вычислить метрики
    # TODO: Построить actual vs predicted plot
    # TODO: Построить residual plot
    pass


def plot_learning_curves(
    model,
    X: pd.DataFrame,
    y: np.ndarray,
    train_sizes: np.ndarray = np.linspace(0.1, 1.0, 10),
    cv: int = 5
) -> None:
    """
    Построить кривые обучения

    Parameters
    ----------
    model : обучаемая модель
    X, y : данные
    train_sizes : np.ndarray
        Доли обучающей выборки
    cv : int
        Число фолдов
    """
    # TODO: Использовать sklearn.model_selection.learning_curve
    # TODO: Построить графики train/validation scores vs sample size
    pass



In [ ]:
# =============================================================================
# ВИЗУАЛИЗАЦИЯ И ОТЧЁТНОСТЬ
# =============================================================================

def plot_residual_diagnostics
    """
    Построить диагностические графики остатков



def create_coefficient_table

    """
    Создать сводную таблицу коэффициентов



def plot_model_comparison_summary

    """
    Визуализировать сравнение моделей по метрикам



def generate_summary_report

    Сгенерировать текстовый отчёт с результатами



In [ ]:
# =============================================================================
# ОСНОВНОЙ ПАЙПЛАЙН
# =============================================================================

def run_full_analysis_pipeline
(
  . . .

) -> Dict:
    """
    Выполнить полный анализ по лабораторной работе

    Этапы:
    1. Загрузка и предобработка данных
    2. OLS модель и диагностика
    .
    .
    .

    Parameters
    ----------
    dataset_name : str
        Название датасета
    target_col : str
        Целевая переменная
   .
   .
   .

    Returns
    -------
    Dict
        Словарь со всеми результатами анализа
    """
    # TODO: Реализовать полный пайплайн согласно требованиям ЛР
    # TODO: Сохранить все графики в output_dir
    # TODO: Сохранить таблицы результатов
    # TODO: Сгенерировать итоговый отчёт
    pass

In [ ]:
# =============================================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# =============================================================================

def set_plot_style() -> None:
    """Установить стиль графиков"""
    plt.style.use('seaborn-v0_8-darkgrid')
    sns.set_palette("husl")
    plt.rcParams['figure.figsize'] = (12, 6)
    plt.rcParams['font.size'] = 10


def save_figure(fig, filename: str, output_dir: str = 'results') -> None:
    """Сохранить фигуру"""
    import os
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(os.path.join(output_dir, filename), dpi=300, bbox_inches='tight')
    plt.close(fig)


def print_section_header(title: str) -> None:
    """Вывести заголовок раздела"""
    print("\n" + "="*80)
    print(f"  {title}")
    print("="*80 + "\n")

In [ ]:
"""
Утилиты для загрузки и предобработки конкретных датасетов
Лабораторная работа 2
"""



# =============================================================================
# NYC TAXI DATA (D1)
# =============================================================================

class NYCTaxiLoader:
    """Загрузчик и обработчик данных NYC Taxi"""

    # Словарь зон TLC -> Borough
    # TODO: Заполнить полный маппинг из taxi_zone_lookup.csv
    ZONE_TO_BOROUGH = {
        # Примеры (не полный список!)
        4: 'Manhattan',
        12: 'Manhattan',
        13: 'Manhattan',
        24: 'Manhattan',
        41: 'Manhattan',
        42: 'Manhattan',
        43: 'Manhattan',
        48: 'Manhattan',
        50: 'Manhattan',
        68: 'Manhattan',
        74: 'Manhattan',
        75: 'Manhattan',
        79: 'Brooklyn',
        80: 'Brooklyn',
        # ... дополнить остальные зоны
    }

    @staticmethod
    def load_nyc_taxi_parquet(
        filepath: str,
        sample_size: Optional[int] = None,
        year: int = 2019,
        month: int = 1
    ) -> pd.DataFrame:
        """
        Загрузить NYC Taxi данные из Parquet

        Parameters
        ----------
        filepath : str
            Путь к .parquet файлу
        sample_size : Optional[int]
            Размер случайной выборки
        year : int
            Год данных (для фильтрации)
        month : int
            Месяц данных (для фильтрации)

        Returns
        -------
        pd.DataFrame
        """
        # TODO: Загрузить parquet
        # df = pd.read_parquet(filepath)

        # TODO: Фильтровать по дате если нужно
        # df = df[(df['pickup_datetime'].dt.year == year) &
        #         (df['pickup_datetime'].dt.month == month)]

        # TODO: Случайная выборка
        # if sample_size and len(df) > sample_size:
        #     df = df.sample(n=sample_size, random_state=42)

        pass

    @staticmethod
    def clean_nyc_taxi(df: pd.DataFrame) -> pd.DataFrame:
        """
        Очистить данные от аномалий

        Parameters
        ----------
        df : pd.DataFrame
            Сырые данные

        Returns
        -------
        pd.DataFrame
            Очищенные данные
        """
        df_clean = df.copy()

        # TODO: Удалить аномальные значения
        # - Негативные или нулевые fare_amount
        # df_clean = df_clean[df_clean['fare_amount'] > 0]

        # - Негативные или слишком большие trip_distance
        # df_clean = df_clean[(df_clean['trip_distance'] > 0) &
        #                     (df_clean['trip_distance'] < 100)]

        # - Негативные чаевые
        # df_clean = df_clean[df_clean['tip_amount'] >= 0]

        # - Аномально большие суммы
        # df_clean = df_clean[df_clean['total_amount'] < 500]

        # - Пассажиры: 1-6
        # df_clean = df_clean[(df_clean['passenger_count'] >= 1) &
        #                     (df_clean['passenger_count'] <= 6)]

        # TODO: Удалить поездки с аномальной длительностью
        # df_clean['duration_min'] = (df_clean['tpep_dropoff_datetime'] -
        #                             df_clean['tpep_pickup_datetime']).dt.total_seconds() / 60
        # df_clean = df_clean[(df_clean['duration_min'] > 1) &
        #                     (df_clean['duration_min'] < 180)]

        return df_clean

    @staticmethod
    def add_temporal_features(df: pd.DataFrame, datetime_col: str = 'tpep_pickup_datetime') -> pd.DataFrame:
        """
        Добавить временные признаки

        Parameters
        ----------
        df : pd.DataFrame
            Данные
        datetime_col : str
            Название колонки с datetime

        Returns
        -------
        pd.DataFrame
        """
        df = df.copy()

        # TODO: Извлечь компоненты времени
        # df['hour'] = df[datetime_col].dt.hour
        # df['day_of_week'] = df[datetime_col].dt.dayofweek
        # df['day_of_month'] = df[datetime_col].dt.day
        # df['month'] = df[datetime_col].dt.month
        # df['year'] = df[datetime_col].dt.year

        # TODO: Категориальные признаки времени
        # df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
        # df['is_rush_hour'] = ((df['hour'].between(7, 9)) |
        #                       (df['hour'].between(17, 19))).astype(int)
        # df['time_of_day'] = pd.cut(df['hour'],
        #                             bins=[0, 6, 12, 18, 24],
        #                             labels=['Night', 'Morning', 'Afternoon', 'Evening'])

        return df

    @staticmethod
    def add_borough_feature(df: pd.DataFrame, zone_col: str = 'PULocationID') -> pd.DataFrame:
        """
        Добавить признак Borough из LocationID

        Parameters
        ----------
        df : pd.DataFrame
            Данные
        zone_col : str
            Название колонки с LocationID

        Returns
        -------
        pd.DataFrame
        """
        df = df.copy()

        # TODO: Маппинг через ZONE_TO_BOROUGH
        # df['borough'] = df[zone_col].map(NYCTaxiLoader.ZONE_TO_BOROUGH)
        # df['borough'] = df['borough'].fillna('Unknown')

        return df

    @staticmethod
    def compute_trip_duration(df: pd.DataFrame) -> pd.DataFrame:
        """
        Вычислить длительность поездки

        Parameters
        ----------
        df : pd.DataFrame
            Данные с pickup и dropoff datetime

        Returns
        -------
        pd.DataFrame
        """
        df = df.copy()

        # TODO: Вычислить в минутах
        # df['trip_duration'] = (df['tpep_dropoff_datetime'] -
        #                        df['tpep_pickup_datetime']).dt.total_seconds() / 60

        return df


# =============================================================================
# OLIST E-COMMERCE DATA (D3)
# =============================================================================

class OlistLoader:
    """Загрузчик и обработчик данных Olist"""

    @staticmethod
    def load_olist_tables(data_dir: str) -> Dict[str, pd.DataFrame]:
        """
        Загрузить все таблицы Olist

        Parameters
        ----------
        data_dir : str
            Директория с CSV файлами

        Returns
        -------
        Dict[str, pd.DataFrame]
            Словарь таблиц
        """
        tables = {}

        # TODO: Загрузить каждую таблицу
        # tables['orders'] = pd.read_csv(f'{data_dir}/olist_orders_dataset.csv')
        # tables['order_items'] = pd.read_csv(f'{data_dir}/olist_order_items_dataset.csv')
        # tables['order_payments'] = pd.read_csv(f'{data_dir}/olist_order_payments_dataset.csv')
        # tables['products'] = pd.read_csv(f'{data_dir}/olist_products_dataset.csv')
        # tables['sellers'] = pd.read_csv(f'{data_dir}/olist_sellers_dataset.csv')
        # tables['customers'] = pd.read_csv(f'{data_dir}/olist_customers_dataset.csv')
        # tables['geolocation'] = pd.read_csv(f'{data_dir}/olist_geolocation_dataset.csv')
        # tables['product_category_name_translation'] = pd.read_csv(
        #     f'{data_dir}/product_category_name_translation.csv'
        # )

        return tables

    @staticmethod
    def merge_olist_data(tables: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        """
        Объединить таблицы Olist в единый датафрейм

        Parameters
        ----------
        tables : Dict[str, pd.DataFrame]
            Словарь загруженных таблиц

        Returns
        -------
        pd.DataFrame
            Объединённые данные на уровне заказа
        """
        # TODO: Начать с orders
        # df = tables['orders'].copy()

        # TODO: Join order_items
        # df = df.merge(tables['order_items'], on='order_id', how='left')

        # TODO: Join payments (агрегировать по order_id)
        # payments_agg = tables['order_payments'].groupby('order_id').agg({
        #     'payment_value': 'sum',
        #     'payment_installments': 'mean'
        # }).reset_index()
        # df = df.merge(payments_agg, on='order_id', how='left')

        # TODO: Join products
        # df = df.merge(tables['products'], on='product_id', how='left')

        # TODO: Join sellers
        # df = df.merge(tables['sellers'], on='seller_id', how='left')

        # TODO: Translate category names
        # df = df.merge(tables['product_category_name_translation'],
        #               on='product_category_name', how='left')

        pass

    @staticmethod
    def compute_olist_features(df: pd.DataFrame) -> pd.DataFrame:
        """
        Вычислить производные признаки для Olist

        Parameters
        ----------
        df : pd.DataFrame
            Объединённые данные

        Returns
        -------
        pd.DataFrame
        """
        df = df.copy()

        # TODO: GMV (Gross Merchandise Value)
        # df['gmv'] = df.groupby('order_id')['price'].transform('sum')

        # TODO: Delivery time
        # df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
        # df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
        # df['delivery_time'] = (df['order_delivered_customer_date'] -
        #                        df['order_purchase_timestamp']).dt.days

        # TODO: Delivery delay
        # df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])
        # df['delivery_delay'] = (df['order_delivered_customer_date'] -
        #                         df['order_estimated_delivery_date']).dt.days

        # TODO: Number of items per order
        # df['num_items'] = df.groupby('order_id')['order_item_id'].transform('count')

        return df


# =============================================================================
# MOVIELENS DATA (D4)
# =============================================================================

class MovieLensLoader:
    """Загрузчик и обработчик данных MovieLens"""

    @staticmethod
    def load_movielens_100k(data_dir: str) -> Dict[str, pd.DataFrame]:
        """
        Загрузить MovieLens 100K данные

        Parameters
        ----------
        data_dir : str
            Директория с файлами

        Returns
        -------
        Dict[str, pd.DataFrame]
            Словарь таблиц
        """
        tables = {}

        # TODO: Загрузить ratings (u.data)
        # columns_ratings = ['user_id', 'item_id', 'rating', 'timestamp']
        # tables['ratings'] = pd.read_csv(
        #     f'{data_dir}/u.data',
        #     sep='\t',
        #     names=columns_ratings
        # )

        # TODO: Загрузить movies (u.item)
        # columns_movies = ['item_id', 'title', 'release_date', 'video_release_date',
        #                   'imdb_url'] + [f'genre_{i}' for i in range(19)]
        # tables['movies'] = pd.read_csv(
        #     f'{data_dir}/u.item',
        #     sep='|',
        #     names=columns_movies,
        #     encoding='latin-1'
        # )

        # TODO: Загрузить users (u.user)
        # columns_users = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
        # tables['users'] = pd.read_csv(
        #     f'{data_dir}/u.user',
        #     sep='|',
        #     names=columns_users
        # )

        return tables

    @staticmethod
    def merge_movielens_data(tables: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        """
        Объединить таблицы MovieLens

        Parameters
        ----------
        tables : Dict[str, pd.DataFrame]
            Словарь таблиц

        Returns
        -------
        pd.DataFrame
            Объединённые данные
        """
        # TODO: Начать с ratings
        # df = tables['ratings'].copy()

        # TODO: Join movies
        # df = df.merge(tables['movies'], on='item_id', how='left')

        # TODO: Join users
        # df = df.merge(tables['users'], on='user_id', how='left')

        pass

    @staticmethod
    def extract_movie_year(df: pd.DataFrame, title_col: str = 'title') -> pd.DataFrame:
        """
        Извлечь год релиза из названия фильма

        Parameters
        ----------
        df : pd.DataFrame
            Данные с колонкой title
        title_col : str
            Название колонки с заголовком

        Returns
        -------
        pd.DataFrame
        """
        df = df.copy()

        # TODO: Извлечь год из названия (обычно в конце в скобках)
        # df['year'] = df[title_col].str.extract(r'\((\d{4})\)')
        # df['year'] = pd.to_numeric(df['year'], errors='coerce')

        return df

    @staticmethod
    def create_genre_features(df: pd.DataFrame) -> pd.DataFrame:
        """
        Создать one-hot признаки жанров

        Parameters
        ----------
        df : pd.DataFrame
            Данные с колонками genre_*

        Returns
        -------
        pd.DataFrame
        """
        # TODO: Жанры уже в one-hot формате в u.item
        # Можно переименовать для удобства

        return df

    @staticmethod
    def compute_user_movie_stats(df: pd.DataFrame) -> pd.DataFrame:
        """
        Вычислить агрегированные статистики пользователей и фильмов

        Parameters
        ----------
        df : pd.DataFrame
            Данные с рейтингами

        Returns
        -------
        pd.DataFrame
        """
        df = df.copy()

        # TODO: Leave-one-out средний рейтинг фильма
        # movie_ratings = df.groupby('item_id')['rating'].transform('sum')
        # movie_counts = df.groupby('item_id')['rating'].transform('count')
        # df['avg_movie_rating_loo'] = (movie_ratings - df['rating']) / (movie_counts - 1)

        # TODO: Активность пользователя
        # df['user_activity'] = df.groupby('user_id')['rating'].transform('count')

        return df


# =============================================================================
# UK ROAD SAFETY DATA - STATS19 (D5)
# =============================================================================

class STATS19Loader:
    """Загрузчик и обработчик данных STATS19"""

    @staticmethod
    def load_stats19_csv(filepath: str, year: int = 2017) -> pd.DataFrame:
        """
        Загрузить STATS19 данные

        Parameters
        ----------
        filepath : str
            Путь к CSV файлу
        year : int
            Год данных

        Returns
        -------
        pd.DataFrame
        """
        # TODO: Загрузить accidents
        # df = pd.read_csv(filepath)

        # TODO: Фильтровать по году если нужно
        # df['date'] = pd.to_datetime(df['date'])
        # df = df[df['date'].dt.year == year]

        pass

    @staticmethod
    def create_severity_score(df: pd.DataFrame) -> pd.DataFrame:
        """
        Создать суррогат серьёзности ДТП

        Формула: 10 * fatal + 3 * serious + 1 * slight

        Parameters
        ----------
        df : pd.DataFrame
            Данные с casualty columns

        Returns
        -------
        pd.DataFrame
        """
        df = df.copy()

        # TODO: Вычислить взвешенную сумму пострадавших
        # df['severity_score'] = (
        #     10 * df['number_of_casualties_fatal'] +
        #     3 * df['number_of_casualties_serious'] +
        #     1 * df['number_of_casualties_slight']
        # )

        return df

    @staticmethod
    def encode_categorical_stats19(df: pd.DataFrame) -> pd.DataFrame:
        """
        Закодировать категориальные признаки

        Parameters
        ----------
        df : pd.DataFrame
            Сырые данные

        Returns
        -------
        pd.DataFrame
        """
        df = df.copy()

        # TODO: Weather conditions
        # weather_mapping = {
        #     1: 'Fine',
        #     2: 'Raining',
        #     3: 'Snowing',
        #     4: 'Fine + high winds',
        #     5: 'Raining + high winds',
        #     6: 'Snowing + high winds',
        #     7: 'Fog or mist',
        #     8: 'Other',
        #     9: 'Unknown'
        # }
        # df['weather_conditions'] = df['weather_conditions'].map(weather_mapping)

        # TODO: Light conditions
        # TODO: Road type
        # TODO: Junction detail

        return df


# =============================================================================
# CHICAGO TAXI DATA (D2)
# =============================================================================

class ChicagoTaxiLoader:
    """Загрузчик и обработчик данных Chicago Taxi"""

    @staticmethod
    def load_chicago_taxi_csv(filepath: str, sample_size: Optional[int] = None) -> pd.DataFrame:
        """
        Загрузить Chicago Taxi данные

        Parameters
        ----------
        filepath : str
            Путь к CSV файлу
        sample_size : Optional[int]
            Размер выборки

        Returns
        -------
        pd.DataFrame
        """
        # TODO: Загрузить данные
        # df = pd.read_csv(filepath)

        # TODO: Парсинг дат
        # df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
        # df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])

        # TODO: Случайная выборка
        # if sample_size and len(df) > sample_size:
        #     df = df.sample(n=sample_size, random_state=42)

        pass

    @staticmethod
    def clean_chicago_taxi(df: pd.DataFrame) -> pd.DataFrame:
        """
        Очистить данные Chicago Taxi

        Parameters
        ----------
        df : pd.DataFrame
            Сырые данные

        Returns
        -------
        pd.DataFrame
        """
        df_clean = df.copy()

        # TODO: Удалить аномалии
        # - Негативные fare
        # - Нулевые trip_miles/trip_seconds
        # - Аномально большие значения

        return df_clean